In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_name = '20000'

data_file = f'../{data_name}_vectors_drags_lifts.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_19993,dim_19994,dim_19995,dim_19996,dim_19997,dim_19998,dim_19999,dim_20000,drag,lift
406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.395,0.425
54,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.645,0.885
241,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.446,0.756
952,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384,0.177
337,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.381,0.074


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    837.000000
mean       0.445233
std        0.112500
min        0.051000
25%        0.369000
50%        0.427000
75%        0.513000
max        0.944000
Name: drag, dtype: float64


In [3]:
import os
save_path = f'./agModels-{data_name}_{label}'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
	Consider setting `time_limit` to ensure training finishes within an expected durat

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_19992,dim_19993,dim_19994,dim_19995,dim_19996,dim_19997,dim_19998,dim_19999,dim_20000,lift
1046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.346
657,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.654
127,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052
951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604
134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.199


In [5]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_all_parts.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-20000_drag/predictor.pkl
Loading: ./agModels-20000_drag/learner.pkl
Loading: ./agModels-20000_drag/models/trainer.pkl
Loading: ./agModels-20000_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/LightGBM

0.48809921741485596
0.30574601888656616
0.42793893814086914
0.4289739429950714
0.5076804757118225
0.3328140377998352
0.38269495964050293
0.38874897360801697
0.394889771938324
0.4085438847541809
0.5656227469444275
0.5552471876144409
0.4062187075614929
0.33931446075439453
0.5332083702087402
0.5483804941177368
0.33647745847702026
0.5384477376937866
0.5850411057472229
0.5392671823501587
0.46349287033081055
0.45387792587280273
0.6754955053329468
0.451079398393631
0.41552379727363586
0.4015202522277832
0.5648825764656067
0.3970033824443817
0.6178740859031677
0.33453452587127686
0.4113047420978546
0.4595773220062256
0.4318740963935852
0.47758054733276367
0.39106494188308716
0.3235955834388733
0.43314141035079956
0.3847835063934326
0.374068945646286
0.40501391887664795
0.3642192780971527
0.3763508200645447
0.6375269889831543
0.41834163665771484
0.5521577596664429
0.3836292624473572
0.38659149408340454
0.5082060694694519
0.40648722648620605
0.3260384500026703
0.5462102890014648
0.45336821675300

Loading: ./agModels-20000_drag/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetFastAI_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/XGBoost_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetTorch_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/WeightedEnsemble_L3/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMXT_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/LightGBM_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/RandomForestMSE_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/CatBoost_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/ExtraTreesMSE_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetFastAI_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/XGBoost_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetTorch_BAG_L3/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMLarge_

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3  -0.069308      67.692911   9080.294354                0.002377           0.422863            3       True         22
1      WeightedEnsemble_L4  -0.069502     130.985360  10621.628307                0.001399           0.474672            4       True         32
2      WeightedEnsemble_L2  -0.069769       6.543130    579.620413                0.002369           0.547630            2       True         12
3        LightGBMXT_BAG_L2  -0.069772      60.330084   8660.420071                2.272196          60.666638            2       True         13
4          LightGBM_BAG_L3  -0.070471     116.964250  10239.046520                2.212428          50.317358            3       True         24
5     ExtraTreesMSE_BAG_L3  -0.070563     123.978739  10266.578847  

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
Loading: ./agModels-20000_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/XGBoost_BAG_L1/model.pkl

                     model  score_test  score_val  pred_time_test  \
0     LightGBMLarge_BAG_L2   -0.070629  -0.071216       41.766635   
1          LightGBM_BAG_L3   -0.070830  -0.070471       77.112276   
2          LightGBM_BAG_L2   -0.071179  -0.070896       41.948718   
3      WeightedEnsemble_L3   -0.071392  -0.069308       50.612104   
4          LightGBM_BAG_L4   -0.071915  -0.072932      115.639313   
5        LightGBMXT_BAG_L2   -0.072026  -0.069772       42.030731   
6      WeightedEnsemble_L2   -0.072092  -0.069769        6.777457   
7        LightGBMXT_BAG_L4   -0.072092  -0.072786      115.754101   
8      WeightedEnsemble_L4   -0.072148  -0.069502       83.997993   
9        LightGBMXT_BAG_L3   -0.072157  -0.070985       77.225690   
10         CatBoost_BAG_L2   -0.072172  -0.070657       54.921692   
11         CatBoost_BAG_L1   -0.072321  -0.072104       17.375677   
12         CatBoost_BAG_L3   -0.072405  -0.071206       92.943492   
13     WeightedEnsemble_L5   -0.07

In [6]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', [])     :     1 | ['lift']
('int', ['bool']) : 19054 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [7]:
train_data_pred = predictor.predict(train_data)
test_data_pred = predictor.predict(test_data)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt(f'./{data_name}_vectors_y_test_hat_{label}.csv', test_data_pred, delimiter=',')
np.savetxt(f'./{data_name}_vectors_y_train_hat_{label}.csv', train_data_pred, delimiter=',')

Loading: ./agModels-20000_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNetFastAI_BAG_L2/model.pkl
Loading: ./agModels-20000_drag/models/NeuralNet